# Método de Montecarlo, versión simple

[**Juan Gómez Romero**](https://decsai.ugr.es/~jgomez)  
Departamento de Ciencias de la Computación e Inteligencia Artificial  
Universidad de Granada  
This work is licensed under the [GNU General Public License v3.0](https://choosealicense.com/licenses/gpl-3.0/).

---

Ejemplo basado en una versión simplificada del [robot navegador](https://github.com/jgromero/eci2019-DRL/blob/master/Tema%203%20-%20Aprendizaje%20por%20Refuerzo/Aprendizaje%20por%20refuerzo.pdf) del curso.

Código adaptado de:
> Udacity (2019) Deep Reinforcement Learning Course. Available in [GitHub](https://github.com/udacity/deep-reinforcement-learning/tree/master/monte-carlo).

# Método de Montecarlo

### Algoritmo
A continuación se proporciona una implementación del algoritmo de Montecarlo para el robot navegador simplificado.

Se generan dos episodios manualmente.

![](navigator_simple.png)

Se requiere [`pandas`](https://pandas.pydata.org) y [`tabulate`](https://pypi.org/project/tabulate/).

In [1]:
!pip install pandas tabulate

You are using pip version 18.0, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import sys
import gym
import numpy as np
from collections import defaultdict
from tabulate import tabulate
import pandas as pd

def generate_custom_episode(ep_number=1):
    """Generar episodio personalizado:
    Params
    ======
    ep_number: numero de episodio a generar (1, 2)
    """
    episode = []
    
    if ep_number==1:
        episode = [
            ((1, 0), 0, -5),
            ((0, 0), 1, +50)
        ]
    else:
        episode = [
            ((1, 0), 1, -1),
            ((1, 1), 3, -1),
            ((1, 0), 1, -1),
            ((1, 1), 0, +50),
        ]
    
    return episode

def print_Q_table(Q):
    """Imprimir tabla Q"""  
    Q_df = pd.DataFrame.from_dict(Q)
    Q_df = Q_df.reindex(sorted(Q_df.columns), axis=1)
    print(tabulate(Q_df.T, headers='keys'))
    
def mc_prediction_q(env, num_episodes, generate_episode, gamma=1.0):
    """Algoritmo de Montecarlo:
    Params
    ======
    env: entorno, definido como estructura de datos
    num_episodes: numero de episodios para explorar
    generate_episode: funcion para generar episodios
    gamma: descuento
    """

    # inicializar diccionarios (prediccion every-visit)
    Q = defaultdict(lambda: np.zeros(env['action_space.n']))  # Q
    N = defaultdict(lambda: np.zeros(env['action_space.n']))  # numero de visitas a (estado, accion)
    returns_sum = defaultdict(lambda: np.zeros(env['action_space.n'])) # suma de recompensa en (estado, accion)
    
    # bucle de episodios
    for i_episode in range(1, num_episodes+1):
        
        # generar episodio
        episode = generate_episode(i_episode)
        
        # actualizar tabla Q
        # - obtener estados, acciones y recompensas del episodio por separado
        states, actions, rewards = zip(*episode)                
        # - obtener gamma para aplicar descuentos
        discounts = np.array([gamma**i for i in range(len(rewards)+1)])        
        # - actualizar suma de recompensa, numero de visitas y Q para cada (estado, accion) del episodio        
        for i, state in enumerate(states):
            returns_sum[state][actions[i]] += sum(rewards[i:]*discounts[:-(1+i)])
            N[state][actions[i]] += 1.0
            Q[state][actions[i]] = returns_sum[state][actions[i]] / N[state][actions[i]]
            
        # imprimir tabla Q
        print("\nEpisodio {:d}:".format(i_episode))
        print_Q_table(Q)
    
    policy = dict((k,np.argmax(v)) for k, v in Q.items())
            
    return Q, policy

Para llamar al algoritmo, creamos el entorno y llamamos a la función `mc_prediction_q`. 

In [3]:
# crear entorno
env = {
    'action_space.n' : 4
}

# obtener estimacion de Q (funcion accion-valor)
Q, policy = mc_prediction_q(env, 2, generate_custom_episode, 0.9)


Episodio 1:
          0    1    2    3
------  ---  ---  ---  ---
(0, 0)    0   50    0    0
(1, 0)   40    0    0    0

Episodio 2:
          0      1    2     3
------  ---  -----  ---  ----
(0, 0)    0  50       0   0
(1, 0)   40  38.87    0   0
(1, 1)   50   0       0  38.6
